In [1]:
# import that packages... 
import mne
import mne.io
import mne_bids
import numpy as np
import plotly.express as px
import q1k_sync_tools as qst

In [2]:
# set some common parameters...
subject_id = '0006' #this is not used for anything right now..
task_code = 'plr' # this is currently used to determine the task specific sync event sequence.. options: as, fsp, go, mmn, nsp, plr, rest, ssaep, ssvep, vs
bids_root = '.' # this will eventually determine the root directory for the BIDS subject folders

In [3]:
#read EEG mff data and convert it to a dataframe..
eeg_raw = mne.io.read_raw_egi('sourcedata/s06_v2/s06_v2_eeg/s06_v2_plr_20230404_022723.mff/',preload=True)
eeg_raw_fresh=eeg_raw.copy() #make a fresh copy for later
eeg_raw_df = eeg_raw.to_data_frame()


Reading EGI MFF Header from /home/james/q1k/q1k_eeget_init/sourcedata/s06_v2/s06_v2_eeg/s06_v2_plr_20230404_022723.mff...
    Reading events ...
    Assembling measurement info ...
    Synthesizing trigger channel "STI 014" ...
    Excluding events {} ...
Reading 0 ... 342605  =      0.000 ...   342.605 secs...


In [6]:
# peak... at the eeg channel types
eeg_types = eeg_raw.copy().get_channel_types()
np.unique(eeg_types)

array(['eeg', 'stim'], dtype='<U4')

In [7]:
#get the EEG events from the stim stim channel
eeg_events = mne.find_events(eeg_raw, shortest_event = 1)

#create the event dictionary..
eeg_event_dict = qst.get_event_dict(eeg_raw,eeg_events)

102 events found
Event IDs: [1 2 3 4 5]


In [8]:
# handle task specific EEG event interpretation..
eeg_events, eeg_stims, eeg_iti, eeg_event_dict = qst.eeg_event_test(eeg_events,eeg_event_dict,task_name=task_code)

Number of stimulus onset DIN events: 32


In [ ]:
# peak... at the EEG event scatter plot.. event time stamp by label index
fig=px.scatter(x=eeg_events[:,0],y=eeg_events[:,2])
fig.show()

In [10]:
#read the asc eye tracking data and convert it to a dataframe...
et_raw = mne.io.read_raw_eyelink ('sourcedata/s06_v2/s06_v2_eyetracking/s06v2plr.asc')
et_raw_fresh=et_raw.copy() #make a fresh copy for later
et_raw_df = et_raw.to_data_frame()

#get the events from the annotation structure
et_annot_events, et_annot_event_dict = mne.events_from_annotations(et_raw)

Loading sourcedata/s06_v2/s06_v2_eyetracking/s06v2plr.asc
Pixel coordinate data detected.
Pass `scalings=dict(eyegaze=1e3)` when using plot method to make traces more legible.
Pupil-size area reported.
There are 4 recording blocks in this file. Times between blocks will be annotated with bad_rec_gap.
Used Annotations descriptions: ['FIX', 'ISI', 'STIM', 'TRACKER_TIME 1 4934305.249', 'TRACKER_TIME 2 4997318.51', 'TRACKER_TIME 3 5057933.263', 'TRACKER_TIME 4 5117997.289', 'TRACKER_TIME 5 5182711.548', 'blink_R', 'fixation_R', 'saccade_R']


In [11]:
# peak... at the ET channel types
et_types = et_raw.get_channel_types()
np.unique(et_types)

array(['eyegaze', 'pupil', 'stim'], dtype='<U7')

In [12]:
#handle task specific ET event interpretation..
et_event_raw_df, et_events, et_stims, et_iti = qst.et_event_test(et_raw_df,task_name=task_code)

Number of eye-tracking stimulus onset DIN events: 32


In [13]:
# peak... at the ET event scatter plot.. event time stamp by DIN index
fig=px.scatter(x=et_events['index'],y=et_events['DIN_diff'])
fig.show()

In [9]:
# peak... at the time offset between the matched events in the EEG and ET recording 
qst.show_sync_offsets(eeg_stims,et_stims)

In [14]:
# combine the EEG and ET recordings given the matched event times.. then create a dataframe copy..
eeg_et_raw = qst.eeg_et_combine(eeg_raw_fresh, et_raw_fresh, eeg_stims, et_stims)
eeg_et_raw_df = eeg_et_raw.to_data_frame()

Zero order coefficient: -53.549851312621485 
First order coefficient: 1.000009677916928
Linear correlation computed as R=1.000 and p=1.23e-161
Drift rate: 9.7 μs/s (total drift over 289.1 s recording: 2.8 ms)
Cropping -53.550 s from the start of other
Resampling other
100 events found
Event IDs: [1 2 3 4]
100 events found
Event IDs: [1 2 3 4]
Cropping 0.035 s from the end of raw
Creating RawArray with float64 data, n_channels=139, n_times=289059
    Range : 0 ... 289058 =      0.000 ...   289.058 secs
Ready.


In [17]:
#get the EEG/ET events from the stim stim channel
eeg_et_events = mne.find_events(eeg_et_raw, shortest_event = 1)

#create the EEG/ETevent dictionary..
eeg_et_event_dict = qst.get_event_dict(eeg_et_raw,eeg_et_events)

100 events found
Event IDs: [1 2 3 4]


In [12]:
#peak... at the EEG/ET
eeg_et_event_dict

{'DIN': 1, 'TSYN': 2, 'plro': 3, 'DIN3': 4, 'DIN2': 5, 'VBeg': 6}

In [24]:
# plot the EEG/ET continuous scroll...
scalings=dict(pupil=1e3)
eeg_et_raw.plot(duration=4,scalings=scalings)


In [18]:
# create event list froma  specific event type of interest...
mask = np.isin(eeg_et_events[:,2],[eeg_et_event_dict['plro']])
eeg_et_events = eeg_et_events[mask]

In [20]:
# epoch the continuous data to the events of interest...
eeg_et_epochs = mne.Epochs(eeg_et_raw, eeg_et_events, tmin=-1, tmax=5)

Not setting metadata
32 matching events found
Setting baseline interval to [-1.0, 0.0] s
Applying baseline correction (mode: mean)
0 projection items activated


In [25]:
# plot the EEG/ET epoch scroll
scalings=dict(pupil=1e2)
eeg_et_epochs.plot(n_epochs=5,scalings=scalings)

Using data from preloaded Raw for 32 events and 6001 original time points ...
0 bad epochs dropped
Using data from preloaded Raw for 5 events and 6001 original time points ...
Using data from preloaded Raw for 3 events and 6001 original time points ...


In [26]:
# export the sync'd continuous data to BIDS format... this seems to have an issue with the 'eyegaze' channel type.. errors
bids_path = mne_bids.BIDSPath(subject=subject_id, task=task_code, root=bids_root)
mne_bids.write_raw_bids(eeg_et_raw, bids_path, events=eeg_et_events, event_id=eeg_et_event_dict, overwrite=True, allow_preload=True, format='EDF')

KeyError: 'eyegaze'

In [ ]:
#show the stimulus DIN event inter-trial intervals... this should be replicated in the eye-tracking recording..
fig = px.line(y=eeg_iti)
fig.show()